In [251]:
# import ALL THE DEPENDENCIES
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import numpy as np
import json
from money import Money
from datetime import datetime
from dateutil.parser import parse

In [252]:
# import the rental data
LA_rental_data1 = pd.read_csv("LARENTALDATA.csv")
LA_rental_data1.head(3)

,Zip Code,City / Community,2019*,2018,2017,2016,2015,2014
0,90001,"Los Angeles (South Los Angeles), Florence-Graham",2351,2325,2202,2046,1836,1763
1,90002,"Los Angeles (Southeast Los Angeles, Watts)",2333,2317,2208,2022,1817,1746
2,90003,"Los Angeles (South Los Angeles, Southeast Los ...",2353,2331,2212,2053,1864,1768


In [253]:
# filter the rental data for LA
LA_rental_price = housing_rent_df[housing_rent_df['County'] == "Los Angeles"].reset_index().drop(columns = ['index'])
LA_rental_price.head(3)

,City Code,City,Metro,County,State,Population Rank,November 2010,December 2010,January 2011,February 2011,...,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017
0,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2184.0,2184.0,2183.0,2188.0,...,2637,2662,2687,2704,2716,2723,2731,2740,2748,2753
1,46298,Long Beach,Los Angeles,Los Angeles,CA,37,1976.0,1971.0,1968.0,1964.0,...,2310,2326,2332,2336,2335,2334,2338,2350,2364,2375
2,45457,Glendale,Los Angeles,Los Angeles,CA,129,2380.0,2385.0,2378.0,2363.0,...,2853,2879,2909,2931,2944,2946,2946,2945,2952,2959


In [254]:
# import the housing listings data
listed_houses_price_df = pd.read_csv("../us-real-estate-listings-by-zip-code/RDC_InventoryCoreMetrics_Zip_Hist.csv",low_memory = False)
listed_houses_price_df.head(3)

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
0,2017-09-01,1001.0,"Agawam, MA",NaN,213700.0,-0.0387,-0.0286,52.0,0.0612,-0.2571,...,0.0,248200.0,-0.0028,0.0973,53.0,0.0600,-0.2535,0.0192,-0.0012,0.0049
1,2017-09-01,1002.0,"Amherst, MA",NaN,358500.0,-0.0465,0.0913,110.0,-0.0678,-0.0351,...,NaN,444400.0,0.0018,0.2396,110.0,-0.0678,-0.0351,0.0000,0.0000,0.0000
2,2017-09-01,1005.0,"Barre, MA",NaN,251300.0,-0.0408,0.1100,24.0,0.1429,-0.1724,...,NaN,250900.0,-0.0260,0.0321,24.0,0.1429,-0.1724,0.0000,0.0000,0.0000


In [255]:
# get all the zipcodes that are referenced in the LA GeoJson file
with open('Zip_Codes_LA_County.geojson') as f:
    data = json.load(f)
    geo_zips = [];
    
for feature in data['features']:
    geo_zips.append(float(feature['properties']['ZIPCODE']))

In [256]:
# function that's going to filter the listing df via a lambda function
def my_rule(row):
    for z in geo_zips:
        if row == z:
            return True
    return False

In [257]:
# LAMMMMBBBDAAAAAA FUNCTIOOOOOOOON
LA_listed_houses = listed_houses_price_df[listed_houses_price_df['ZipCode'].apply(lambda row: my_rule(row))]
LA_listed_houses = LA_listed_houses[LA_listed_houses["Month"] == "2017-09-01"]
LA_listed_houses.head(3)

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
13403,2017-09-01,90001.0,"Los Angeles, CA",*,372300.0,0.0022,NaN,41.0,0.1714,NaN,...,4.0000,381800.0,0.0111,0.1070,51.0,0.2439,0.7586,0.2439,0.0725,0.1698
13404,2017-09-01,90002.0,"Los Angeles, CA",NaN,339700.0,0.0006,0.0760,60.0,-0.0769,0.0526,...,-0.7500,342100.0,0.0130,0.0782,62.0,-0.1268,-0.0462,0.0333,-0.0590,-0.1070
13405,2017-09-01,90003.0,"Los Angeles, CA",NaN,357500.0,-0.0102,0.0949,45.0,0.1538,-0.2623,...,-0.6667,368800.0,-0.0075,0.1386,48.0,0.1163,-0.3143,0.0667,-0.0359,-0.0809


In [258]:
# export the nice data
LA_listed_houses.to_csv("LA_Zip_Listings.csv")

In [259]:
# this cell updates the geojson file to make it easier to reference when making the map with leaflet
for feature in data['features']:
    try:
        med_price = int(LA_listed_houses[LA_listed_houses["ZipCode"] == float(feature['properties']['ZIPCODE'])]["Median Listing Price"])
        rental_price = int(LA_rental_data1[LA_rental_data1["Zip Code"] == float(feature['properties']['ZIPCODE'])]["2017"])
        med_price1 = Money(str(med_price), 'USD').format('en_US')
        rental_nice = Money(str(rental_price), 'USD').format('en_US')
        feature.update({"Median_Price" : med_price1})
        feature.update({"Median_Price_Float" : med_price})
        feature.update({"Median_Rental_Price" : rental_price})
        feature.update({"Median_Rental_Nice" : rental_nice})
    except:
        feature.update({"Median_Price" : "PRICE NOT FOUND"})
        feature.update({"Median_Price_Float" : "NONE"})
        feature.update({"Median_Rental_Price" : "NONE"})
        feature.update({"Median_Rental_Nice" : "NONE"})

In [260]:
# export the data into another json
with open("Zip_Codes_LA_County1.geojson", 'w') as f:
    json.dump(data,f)